In [1]:
# extracts noise model from a real IBM device as superoperators representing a noisy gate set
# It's simplest to run this notebook on IBM Quantum Lab
# Otherwise you have to track down the info necessary for IBMProvider 

In [ ]:
from qiskit_ibm_provider import IBMProvider
from qiskit_aer.noise import NoiseModel
import qiskit.quantum_info as qi
import numpy as np
import pickle 

In [ ]:
# get noise model
provider = IBMProvider()
be = provider.get_backend("ibm_perth")
nm = NoiseModel.from_backend(be)

print(nm)

In [ ]:
# superoperator for noisy cx controlled on top qubit 
cx_top = np.array([[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
exact_cx_top = np.kron(cx_top, cx_top.conj().T)    # the superoperator for a unitary channel, U, is simply (U \otimes U^d)
noise = nm._local_quantum_errors['cx'][(0,1)]
probs, circuits = noise.probabilities, noise.circuits 
noise_op = np.array(sum([probs[i] * qi.SuperOp(circuits[i]) for i in range(len(probs))]))
noisy_cx_top = noise_op @ exact_cx_top

# superoperator for noisy cx controlled on bottom qubit
cx_bottom = np.array([[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]])
exact_cx_bottom = np.kron(cx_bottom, cx_bottom.conj().T)
noise = nm._local_quantum_errors['cx'][(1,0)]
probs, circuits = noise.probabilities, noise.circuits 
noise_op = np.array(sum([probs[i] * qi.SuperOp(circuits[i]) for i in range(len(probs))]))
noisy_cx_bottom = noise_op @ exact_cx_bottom

# superoperator for noisy x
x = np.array([[0,1],[1,0]])
exact_x = np.kron(x,x.conj().T)
noise = nm._local_quantum_errors['x'][(0,)]
probs, circuits = noise.probabilities, noise.circuits 
noise_op = np.array(sum([probs[i] * qi.SuperOp(circuits[i]) for i in range(len(probs))]))
noisy_x = noise_op @ exact_x

# superoperator for noisy sx
sx = (1/2)*np.array([[1+1.j,1-1.j],[1-1.j,1+1.j]])
exact_sx = np.kron(sx,sx.conj().T)
noise = nm._local_quantum_errors['sx'][(0,)]
probs, circuits = noise.probabilities, noise.circuits 
noise_op = np.array(sum([probs[i] * qi.SuperOp(circuits[i]) for i in range(len(probs))]))
noisy_sx = noise_op @ exact_sx

# note that Rz gates are also a part of the IBM gateset but do not accrue errors 
# since they are applied virtually in classical software

In [ ]:
pickle_out = open('ibm_perth_gates.pickle', 'wb')
pickle.dump([exact_cx_top, noisy_cx_top, exact_cx_bottom, noisy_cx_bottom, exact_x, noisy_x, exact_sx, noisy_sx], pickle_out)
pickle_out.close()